### Generates Vectos from unique names from source and creates search indexes

In [30]:
import numpy as np 
import pandas as pd 
import pyphi
import requests
import os
import falconn
import random
from scipy import spatial

In [42]:
def brute_force_query(name, number, names, vectors):
    query = call_embedding_ws(name)
    df = pd.DataFrame(index=range(len(vectors)), columns=['id', 'name', 'cosine'])   
    for idx, vect in enumerate(vectors):
        df['cosine'][idx] = spatial.distance.cosine(query, vect)
    result = df.sort_values(['cosine'], ascending=[False]).head(number)
    for idx, row in result.iterrows():
        row['name'] = names['name'][idx]
    return result

def query_falconn_index(name, lhs_table, number, names, vectors):
    query = call_embedding_ws(name)
    response = lhs_table.find_k_nearest_neighbors(query[0], k=number)
    
    return process_lhs_table_response(query, response, names, vectors)

def process_lhs_table_response(query, response, names, vectors):
    df = pd.DataFrame(index=range(len(response)), columns=['id', 'name', 'cosine'])   
    i = 0
    
    for resp in response:
        name = names.get_value(resp, 'name')
        cos = cos_similarity(query, vectors[resp])
        df.set_value(index=i, col='id', value=resp)
        df.set_value(index=i, col='name', value=name)
        df.set_value(index=i, col='cosine', value=cos)
        i = i + 1
     
    df = df.sort_values(by='cosine', ascending=False)
    df = df.reset_index(drop=True)

    return df

def cos_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)
    return dot_product

def call_embedding_ws(names):
    names = [item.lower() for item in names]
    
#    url = 'http://54.36.53.127:8009/embedding'
    url = 'http://127.0.0.1:8009/embedding'
    headers = {"content-type": "application/json"}
    response = requests.post(url, json=names)
    
    arr = pyphi.jsonify.loads(response.text)
    x = np.array(arr)
    x = x.astype(dtype=np.float32)
    
    return x

input_path = 'data/src_distinct_names/src'
output_path = 'data/src_distinct_names'

#### Calculate vectors for names in source files to output files

In [ ]:
%%time
directory = os.fsencode(input_path)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".csv"): 
        print(filename)
        names = pd.read_csv(input_path + '/' + filename, sep='|', names=['name'], header=None)
#        np.where(pd.isnull(names))
#        names.iloc[245193]
        
        i = -1
        j = -1
        ws_params = []

        file = open(output_path + '/vectors_' + filename,"w") 

        for index, row in names.iterrows():
            ws_params.append(row['name'].strip().lower())
            i = i + 1
            if i % 512 == 0:
                ws_result = call_embedding_ws(ws_params)
                for vect in ws_result:
                    j = j + 1
                   
                    v_str = np.array2string(vect, separator=',', max_line_width=10000).strip("[]").replace(" ", "")
                    
                    #print("{n}|{v}\n".format(n=names['name'][j], v=vect[:5]))
                    file.write("{n}|{v}\n".format(n=names['name'][j], v=v_str))
                ws_params = []
                print(i)

        if len(ws_params) > 0:
            ws_result = call_embedding_ws(ws_params)
            for vect in ws_result:
                j = j + 1
                v_str = np.array2string(vect, separator=',', max_line_width=10000).strip("[]").replace(" ", "")
                file.write("{n}|{v}\n".format(n=names['name'][j], v=v_str))

        file.close()

In [3]:
%%time
directory = os.fsencode(input_path)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".csv"): 
        print(filename)
        names = pd.read_csv(input_path + '/' + filename, sep='|', names=['name'], header=None)
#        np.where(pd.isnull(names))
#        names.iloc[245193]
        
        ws_params = []
        file = open(output_path + '/vectors_' + filename,"w") 

        for index, row in names.iterrows():
            ws_params.append(row['name'].strip().lower())
            ws_result = call_embedding_ws(ws_params)
            v_str = np.array2string(ws_result[0], separator=',', max_line_width=10000).strip("[]").replace(" ", "")
            file.write("{n}|{v}\n".format(n=row['name'], v=v_str))
            ws_params = []
        file.close()

family_names_master_arb_distinct.csv
family_names_master_eng_distinct.csv
given_names_dan_arb_distinct.csv
given_names_dan_eng_distinct.csv
given_names_master_arb_distinct.csv
given_names_master_eng_distinct.csv
CPU times: user 1h 11min 44s, sys: 1min 47s, total: 1h 13min 31s
Wall time: 2h 10min 27s


#### Read vector files and parse vector strings

In [4]:
directory = os.fsencode(output_path)
names_dict = {}
vectors_dict = {}
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".csv"): 
        print(filename)
        
        names_dict[filename] = pd.read_csv(output_path + '/' + filename, sep='|', names=['name', 'vector_string'], header=None)
        
        vector_strings = names_dict[filename]['vector_string'].as_matrix()
        vectors = np.zeros(shape=(len(vector_strings),256))
        i = 0
        for v in vector_strings:
            try:
                x = np.fromstring(v, dtype=np.float32, sep=',')
                vectors[i] = x
                i = i+1
            except:
                print("{i} {v}".format(i=i, v=v))
                raise
            
        vectors_dict[filename] = vectors.astype(dtype=np.float32)

vectors_given_names_master_eng_distinct.csv
vectors_family_names_master_arb_distinct.csv
vectors_family_names_master_eng_distinct.csv
vectors_given_names_dan_arb_distinct.csv
vectors_given_names_dan_eng_distinct.csv
vectors_given_names_master_arb_distinct.csv


#### Merge english and arabic names separately

In [5]:
eng_names_merged =  pd.DataFrame(columns=['name', 'vector_string'])
arb_names_merged = pd.DataFrame(columns=['name', 'vector_string'])


for key, value in names_dict.items():
    if 'eng' in key:
        eng_names_merged = eng_names_merged.append(value).reset_index(drop=True)

eng_names_merged = eng_names_merged.drop_duplicates(['name']).reset_index(drop=True)
eng_names_merged = eng_names_merged.reset_index(drop=True)
        
vector_strings = eng_names_merged['vector_string'].as_matrix()
eng_vectors_merged = np.zeros(shape=(len(vector_strings),256))
i = 0
for v in vector_strings:
    x = np.fromstring(v, dtype=np.float32, sep=',')
    eng_vectors_merged[i] = x
    i = i+1
    
eng_vectors_merged = eng_vectors_merged.astype(dtype=np.float32)



for key, value in names_dict.items():
    if 'arb' in key:
        arb_names_merged = arb_names_merged.append(value).reset_index(drop=True)

arb_names_merged = arb_names_merged.drop_duplicates(['name']).reset_index(drop=True)
arb_names_merged = arb_names_merged.reset_index(drop=True)
        
vector_strings = arb_names_merged['vector_string'].as_matrix()
arb_vectors_merged = np.zeros(shape=(len(vector_strings),256))
i = 0
for v in vector_strings:
    x = np.fromstring(v, dtype=np.float32, sep=',')
    arb_vectors_merged[i] = x
    i = i+1
arb_vectors_merged = arb_vectors_merged.astype(dtype=np.float32) 


#### Validate vectors

In [12]:
for s in random.sample(range(0, len(arb_names_merged)), 2000):
    name = arb_names_merged.iloc[s]['name']
    control_vect = call_embedding_ws([name])[0]
    vect = np.fromstring(arb_names_merged.iloc[s]['vector_string'], dtype=np.float32, sep=',')
    test = control_vect != vect
    if test.any == False:
        print(name)
        print(control_vect == vect)
    
for s in random.sample(range(0, len(eng_names_merged)), 2000):
    name = eng_names_merged.iloc[s]['name']
    control_vect = call_embedding_ws([name])[0]
    vect = np.fromstring(eng_names_merged.iloc[s]['vector_string'], dtype=np.float32, sep=',')
    test = control_vect != vect
    if test.any == False:
        print(name)
        print(control_vect == vect)

#### Build Falconn indexes

In [13]:
%%time
number_of_tables = 50
assert eng_vectors_merged.dtype == np.float32
assert arb_vectors_merged.dtype == np.float32

params_cp = falconn.LSHConstructionParameters()
params_cp.dimension = len(eng_vectors_merged[0])
params_cp.lsh_family = falconn.LSHFamily.CrossPolytope
params_cp.distance_function = falconn.DistanceFunction.EuclideanSquared
params_cp.l = number_of_tables
# we set one rotation, since the data is dense enough,
# for sparse data set it to 2
params_cp.num_rotations = 2
params_cp.seed = 5721840
# we want to use all the available threads to set up
params_cp.num_setup_threads = 0
params_cp.storage_hash_table = falconn.StorageHashTable.BitPackedFlatHashTable
# we build 24-bit hashes so that each table has
# 2^24 bins; this is a good choise since 2^24 is of the same
# order of magnitude as the number of data points
falconn.compute_number_of_hash_functions(18, params_cp)

eng_merged_table = falconn.LSHIndex(params_cp)
eng_merged_table.setup(eng_vectors_merged)

eng_merged_query_object = eng_merged_table.construct_query_object()




params_cp = falconn.LSHConstructionParameters()
params_cp.dimension = len(arb_vectors_merged[0])
params_cp.lsh_family = falconn.LSHFamily.CrossPolytope
params_cp.distance_function = falconn.DistanceFunction.EuclideanSquared
params_cp.l = number_of_tables
# we set one rotation, since the data is dense enough,
# for sparse data set it to 2
params_cp.num_rotations = 2
params_cp.seed = 5721840
# we want to use all the available threads to set up
params_cp.num_setup_threads = 0
params_cp.storage_hash_table = falconn.StorageHashTable.BitPackedFlatHashTable
# we build 24-bit hashes so that each table has
# 2^24 bins; this is a good choise since 2^24 is of the same
# order of magnitude as the number of data points
falconn.compute_number_of_hash_functions(18, params_cp)

arb_merged_table = falconn.LSHIndex(params_cp)
arb_merged_table.setup(arb_vectors_merged)

arb_merged_query_object = arb_merged_table.construct_query_object()

CPU times: user 53.3 s, sys: 119 ms, total: 53.4 s
Wall time: 13.9 s


In [159]:
%%time
query_falconn_index(['maria'], eng_merged_query_object, 50, eng_names_merged, eng_vectors_merged)

CPU times: user 29.7 ms, sys: 3.41 ms, total: 33.1 ms
Wall time: 44.5 ms


,id,name,cosine
0,200347,MARIA,[1.0]
1,200345,MARIA,[1.0]
2,200344,MARIA,[1.0]
3,200346,MARIA,[1.0]
4,205406,MARYE,[0.999995]
5,211976,MEREA,[0.999995]
6,199917,MAREY,[0.999993]
7,199778,MAREI,[0.999993]
8,199779,MAREI,[0.999993]
9,205085,MARYA,[0.999991]


In [61]:
%%time
query_falconn_index(['hamza'], arb_merged_query_object, 20, arb_names_merged, arb_vectors_merged)

CPU times: user 22 ms, sys: 3.39 ms, total: 25.4 ms
Wall time: 44.6 ms


,id,name,cosine
0,95803,جمكو,[0.999957]
1,32591,أهمات,[0.999947]
2,182657,شموطى,[0.999924]
3,162879,سميده,[0.999923]
4,95859,جموعى,[0.999922]
5,162880,سميدى,[0.999921]
6,162774,سموده,[0.999918]
7,16251,اكميت,[0.999913]
8,162842,سميثى,[0.999908]
9,162585,سمتي,[0.999908]


In [43]:
%%time
brute_force_query(['hamza'], 10, eng_names_merged, eng_vectors_merged)

         id name    cosine
0       NaN  NaN  0.922534
1       NaN  NaN  0.852089
2       NaN  NaN  0.969452
3       NaN  NaN   0.96863
4       NaN  NaN     1.028
5       NaN  NaN   1.08756
6       NaN  NaN   1.28546
7       NaN  NaN   1.23617
8       NaN  NaN   1.25346
9       NaN  NaN   1.13965
10      NaN  NaN   1.11676
11      NaN  NaN   1.22352
12      NaN  NaN   1.19719
13      NaN  NaN   1.14061
14      NaN  NaN   1.28176
15      NaN  NaN   1.26895
16      NaN  NaN   1.23348
17      NaN  NaN   1.25568
18      NaN  NaN   1.29006
19      NaN  NaN   1.27383
20      NaN  NaN    1.2393
21      NaN  NaN   1.27736
22      NaN  NaN   1.26333
23      NaN  NaN   1.23499
24      NaN  NaN   1.20789
25      NaN  NaN   1.27239
26      NaN  NaN   1.26404
27      NaN  NaN   1.29436
28      NaN  NaN   1.29588
29      NaN  NaN   1.26581
...     ...  ...       ...
388892  NaN  NaN   1.09614
388893  NaN  NaN   1.07321
388894  NaN  NaN  0.969165
388895  NaN  NaN   1.14809
388896  NaN  NaN  0.956963
3

,id,name,cosine
185336,NaN,LOULOUBELLE,1.53778
174744,NaN,LAULAUPEAALU,1.53408
171781,NaN,LADDAVAL,1.51165
187285,NaN,LULUBEL,1.50895
313782,NaN,SOLOZABAL,1.50756
171818,NaN,LADISABEL,1.50489
180334,NaN,LILYBEL,1.50366
70091,NaN,DELIALAP,1.50362
248270,NaN,OLAZABAL,1.50316
162418,NaN,KHILOLBI,1.49989


In [146]:
%%time
target_dir = "/home/jupyter/notebooks/PoC/data-preparation/output/understanding_data/"
top_given_names = pd.read_csv(target_dir + 'all_names_with_yamli.tsv', sep='\t', header=None)

CPU times: user 855 ms, sys: 65.5 ms, total: 920 ms
Wall time: 918 ms


In [134]:
query_falconn_index(['IBRAHIM'], arb_merged_query_object, 20, arb_names_merged, arb_vectors_merged) 

,id,name,cosine
0,317089,يبرايم,[0.998642]
1,1919,إبراهم,[0.998433]
2,2149,ابروجيم,[0.997736]
3,1886,ابراجيم,[0.997695]
4,14576,افروم,[0.996175]
5,2164,إبروهيم,[0.995895]
6,1923,إبراهيم,[0.995816]
7,2146,إبرهيما,[0.995789]
8,14603,افريما,[0.995388]
9,2150,إبروخيم,[0.994896]


In [130]:
%%time
set_len = len(top_given_names)
for index, row in top_given_names.iterrows():
    arb = row[4].split(',')
    arb = [a.lower() for a in arb]
    eng = row[3].split(',')
    eng = [e.lower() for e in eng]

    arb_model_matches = []
    eng_model_matches = []
    for a in arb:
        arb_matches = query_falconn_index([a], arb_merged_query_object, 20, arb_names_merged, arb_vectors_merged)
        arb_matches = arb_matches['name'].tolist()
        arb_matches = [a.strip(' ') for a in arb_matches]
        arb_model_matches.extend(arb_matches)
        eng_matches = query_falconn_index([a], eng_merged_query_object, 20, eng_names_merged, eng_vectors_merged)
        eng_matches = eng_matches['name'].tolist()
        eng_matches = [a.strip(' ') for a in eng_matches]
        eng_model_matches.extend(eng_matches)
    for e in eng:
        arb_matches = query_falconn_index([e], arb_merged_query_object, 20, arb_names_merged, arb_vectors_merged)
        arb_matches = arb_matches['name'].tolist()
        arb_matches = [a.strip(' ') for a in arb_matches]
        arb_model_matches.extend(arb_matches)
        eng_matches = query_falconn_index([e], eng_merged_query_object, 20, eng_names_merged, eng_vectors_merged)
        eng_matches = eng_matches['name'].tolist()
        eng_matches = [a.strip(' ') for a in eng_matches]
        eng_model_matches.extend(eng_matches)
    top_given_names.set_value(index, 'arb_model_matches', ','.join(set(arb_model_matches)))
    top_given_names.set_value(index, 'eng_model_matches', ','.join(set(eng_model_matches)))
    
    if index % 1000 == 0:
        print("processed {i} out of {t}".format(i=index, t=set_len))

processed 0 out of 66323
processed 1000 out of 66323
processed 2000 out of 66323
processed 3000 out of 66323
processed 4000 out of 66323
processed 5000 out of 66323
processed 6000 out of 66323
processed 7000 out of 66323
processed 8000 out of 66323
processed 9000 out of 66323
processed 10000 out of 66323
processed 11000 out of 66323
processed 12000 out of 66323
processed 13000 out of 66323
processed 14000 out of 66323
processed 15000 out of 66323
processed 16000 out of 66323
processed 17000 out of 66323
processed 18000 out of 66323
processed 19000 out of 66323
processed 20000 out of 66323
processed 21000 out of 66323
processed 22000 out of 66323
processed 23000 out of 66323
processed 24000 out of 66323
processed 25000 out of 66323
processed 26000 out of 66323
processed 27000 out of 66323
processed 28000 out of 66323
processed 29000 out of 66323
processed 30000 out of 66323
processed 31000 out of 66323
processed 32000 out of 66323
processed 33000 out of 66323
processed 34000 out of 6632

import csv
top_given_names.to_csv(target_dir + 'all_names_with_yamli.tsv',sep='\t', quoting=csv.QUOTE_NONE, mode = 'w', header=False, index=False)


row = top_given_names.loc[0]

%%time
arb = row[4].split(',')
arb = [a.lower() for a in arb]
eng = row[3].split(',')
eng = [e.lower() for e in eng]

arb_model_matches = []
eng_model_matches = []
for a in arb:
    arb_matches = query_falconn_index([a], arb_merged_query_object, 20, arb_names_merged, arb_vectors_merged)
    arb_matches = arb_matches['name'].tolist()
    arb_matches = [a.strip(' ') for a in arb_matches]
    arb_model_matches.extend(arb_matches)
    eng_matches = query_falconn_index([a], eng_merged_query_object, 20, eng_names_merged, eng_vectors_merged)
    eng_matches = eng_matches['name'].tolist()
    eng_matches = [a.strip(' ') for a in eng_matches]
    eng_model_matches.extend(eng_matches)
for e in eng:
    arb_matches = query_falconn_index([e], arb_merged_query_object, 20, arb_names_merged, arb_vectors_merged)
    arb_matches = arb_matches['name'].tolist()
    arb_matches = [a.strip(' ') for a in arb_matches]
    arb_model_matches.extend(arb_matches)
    eng_matches = query_falconn_index([e], eng_merged_query_object, 20, eng_names_merged, eng_vectors_merged)
    eng_matches = eng_matches['name'].tolist()
    eng_matches = [a.strip(' ') for a in eng_matches]
    eng_model_matches.extend(eng_matches)
print(len(arb_model_matches))
print(len(set(arb_model_matches)))
print(len(eng_model_matches))
print(len(set(eng_model_matches)))
top_given_names.set_value(0, 'arb_model_matches', ','.join(set(arb_model_matches)))
top_given_names.set_value(0, 'eng_model_matches', ','.join(set(eng_model_matches)))

In [141]:
import csv
top_given_names.to_csv(target_dir + 'all_names_with_yamli.tsv',sep='\t', quoting=csv.QUOTE_NONE, mode = 'w', header=False, index=False)


In [ ]:
top_given_names.iloc[4]['eng_model_matches']

In [ ]:
top_given_names

In [154]:
%%time
for index, row in top_given_names.iterrows():
    yamli_varinats = row[5].split(',')
    yamli_varinats = [a.lower() for a in yamli_varinats]
    arb_varinats = row[6].split(',')
    arb_varinats = [a.lower() for a in arb_varinats]
    eng_variants = row[7].split(',')
    eng_variants = [e.lower() for e in eng_variants]
    model_arb_variants = row[8].split(',')
    model_arb_variants = [a.lower() for a in model_arb_variants]
    model_eng_variants = row[8].split(',')
    model_eng_variants = [a.lower() for a in model_eng_variants]
    filtered_yamli_variants = set(yamli_varinats) - set(arb_varinats)
    top_given_names.set_value(index, 'filtered_yamli_variants', ','.join(set(filtered_yamli_variants)))
    print(len(yamli_varinats))
    print(len(filtered_yamli_variants))

180
167
56
54
50
46
43
36
21
19
249
235
56
53
68
65
13
12
15
14
58
57
30
28
39
35
17
16
16
15
30
29
20
19
92
86
32
31
17
15
150
141
25
24
15
12
15
14
18
17
32
30
41
40
12
11
49
47
9
8
409
399
94
91
153
148
6
5
19
18
27
26
120
116
14
13
10
9
19
17
39
36
81
74
28
27
32
30
18
17
13
12
38
36
9
8
50
47
145
139
21
19
32
26
29
28
27
26
22
20
11
10
15
14
18
17
8
7
10
9
18
17
56
55
32
29
19
18
8
7
21
20
28
27
56
50
28
27
12
11
18
16
699
676
12
11
32
30
124
113
24
23
10
9
8
7
8
7
29
27
19
18
19
18
20
19
13
12
14
12
27
26
28
27
31
30
18
16
42
37
35
32
33
32
14
14
44
39
255
239
6
5
14
13
413
393
42
40
47
44
14
13
16
15
10
9
40
38
10
9
27
25
14
12
35
33
11
10
20
19
31
30
15
14
11
10
92
83
22
20
48
43
10
9
20
19
41
40
13
12
39
36
57
54
8
7
436
425
38
37
11
10
18
16
46
42
3
2
20
19
70
69
19
18
29
28
40
39
14
12
13
12
15
14
53
51
38
37
14
12
10
9
25
22
12
11
32
31
14
13
16
16
26
22
10
10
27
26
28
26
77
71
15
14
55
52
10
9
407
396
37
32
8
7
15
14
18
16
27
26
16
15
21
19
12
10
17
16
27
25
6
5
16
15
14
1

9
26
25
8
7
11
10
10
9
10
8
63
56
12
11
17
16
10
9
4
3
8
7
12
10
10
9
22
21
10
9
10
10
4
3
30
29
27
26
8
7
10
9
18
17
55
54
16
15
10
9
12
10
10
9
15
14
8
6
10
10
10
9
13
12
12
10
23
22
15
14
15
14
10
9
39
37
4
3
19
18
3
2
24
23
161
150
10
9
28
27
10
9
10
9
23
22
10
9
15
14
10
9
10
10
10
10
39
37
10
9
12
11
15
15
12
10
10
9
19
18
4
3
10
10
16
15
15
14
10
9
10
9
10
9
23
21
10
9
15
15
20
20
20
19
28
27
10
8
10
9
8
7
16
15
10
9
12
11
20
19
15
15
74
70
14
12
10
10
14
14
22
20
10
9
18
17
18
17
10
9
10
9
23
22
15
15
28
28
17
16
12
11
24
22
20
18
10
9
13
12
10
10
14
13
18
17
14
13
10
9
8
7
10
9
10
10
24
23
19
18
6
5
24
22
10
9
20
19
27
26
18
17
26
25
10
8
10
9
17
16
19
18
21
17
8
7
16
15
15
14
19
18
19
18
14
13
8
8
6
5
9
8
10
10
10
10
10
9
14
13
71
64
12
11
10
10
15
14
16
15
10
10
10
9
10
9
10
9
19
17
10
9
34
33
19
18
23
22
13
12
13
12
25
24
10
10
12
11
10
9
4
3
14
14
10
9
10
9
18
17
24
23
10
9
15
15
39
38
15
14
10
9
10
9
31
30
10
10
14
13
10
9
10
9
10
9
12
11
17
16
14
13
21
21
33
32
10
10
34


10
10
10
10
10
10
10
15
15
10
10
15
14
10
9
10
9
10
9
4
3
6
5
10
10
14
13
8
7
10
10
15
14
10
10
10
9
10
10
21
20
27
26
10
10
10
10
10
9
10
10
10
9
10
10
15
14
8
8
8
7
8
8
10
9
10
9
10
9
10
9
10
9
4
3
34
33
10
10
27
26
10
9
13
12
16
15
10
10
8
7
27
27
10
10
15
14
12
11
10
10
32
31
18
18
10
10
10
9
14
13
18
17
13
12
8
7
9
9
10
9
23
22
8
7
10
10
14
14
21
20
10
10
10
9
10
10
11
10
18
17
10
10
29
28
10
9
10
10
10
9
10
9
15
15
14
13
6
5
10
10
23
22
10
10
10
10
10
10
10
10
8
8
10
9
43
42
10
9
10
10
10
9
30
28
10
10
10
9
15
14
10
10
10
10
10
9
10
9
10
10
10
10
8
7
14
13
10
10
10
9
36
34
15
14
32
31
10
10
12
11
15
15
15
15
14
13
12
11
10
10
10
10
10
10
17
16
18
17
10
10
10
9
9
8
17
17
10
10
15
14
11
10
10
10
10
10
10
9
15
15
10
9
15
14
10
9
10
10
10
9
13
12
10
9
14
13
10
10
22
21
17
16
8
7
15
14
10
10
10
9
10
10
10
9
10
10
12
11
12
11
15
14
17
16
13
12
10
10
10
10
14
14
10
10
34
34
10
9
8
7
16
15
8
7
11
11
10
10
18
18
10
9
10
10
10
10
10
10
13
12
10
10
16
15
10
10
10
10
14
13
10
10
10
10
17
17


22
9
8
11
11
10
9
8
7
10
10
18
17
10
10
10
9
10
10
32
31
8
7
27
25
19
18
10
9
10
9
8
8
10
10
10
9
13
12
10
10
10
10
18
17
10
9
10
10
10
10
9
8
13
13
12
11
10
10
15
15
14
13
10
10
63
60
15
14
18
17
10
10
18
17
10
9
4
3
10
10
10
9
10
10
15
14
12
12
12
11
10
10
10
10
10
10
23
22
15
15
10
9
10
10
10
10
10
9
10
10
15
15
10
10
12
11
15
15
10
9
4
3
10
9
10
9
10
9
22
21
10
9
10
10
50
49
16
15
14
12
10
10
6
5
10
9
16
15
10
10
15
14
16
14
10
9
19
18
10
9
6
5
10
9
11
10
16
15
14
13
10
10
15
14
10
10
13
13
13
12
21
20
10
9
19
18
10
9
36
36
10
9
15
13
10
9
11
11
10
9
10
10
10
10
10
10
10
9
47
43
20
19
10
10
17
16
30
29
15
15
18
17
16
15
10
9
11
10
13
13
16
15
10
9
19
18
8
7
10
9
22
21
30
29
11
10
19
18
11
10
10
9
10
9
14
13
10
9
22
22
16
15
10
10
13
12
10
10
10
10
10
9
14
13
21
20
10
9
13
12
6
5
17
17
83
80
12
11
15
15
31
30
10
9
10
10
10
9
21
20
10
10
18
17
30
29
4
3
15
14
15
14
8
7
10
10
10
9
37
34
4
3
10
10
4
3
10
9
26
25
10
10
10
10
16
16
26
25
10
10
10
10
16
14
12
11
15
15
22
21
4
3
22
21
10
9

12
11
14
14
10
10
10
9
10
10
8
7
10
10
10
10
17
16
6
5
10
10
8
8
10
10
11
11
10
9
11
10
10
9
10
10
10
10
4
3
10
9
10
9
23
23
15
15
10
9
4
3
10
10
16
15
21
20
6
5
19
19
10
10
22
21
26
25
13
12
15
15
10
10
10
9
10
10
40
39
10
9
14
14
17
16
10
10
10
9
8
7
10
9
26
25
10
9
10
10
10
9
15
13
10
10
10
9
10
9
10
9
8
7
12
11
10
10
10
9
10
9
10
10
4
3
10
9
10
9
10
9
14
13
14
14
14
14
10
9
4
4
10
9
10
10
10
10
10
9
10
9
15
15
19
18
13
12
17
16
10
10
10
10
14
14
10
9
10
9
6
5
15
14
10
9
10
9
8
7
10
9
25
24
10
9
15
15
10
9
10
10
6
6
15
14
15
15
15
15
4
4
17
17
15
14
10
10
10
9
8
7
10
10
10
10
8
8
10
10
11
11
10
10
10
9
10
10
16
16
22
21
10
9
11
11
10
9
15
15
10
9
10
9
11
11
10
9
16
15
10
10
10
9
10
10
10
9
10
10
17
17
10
9
10
10
10
10
13
12
10
9
10
9
10
10
14
13
10
10
10
10
8
7
12
11
6
5
10
9
15
15
6
5
11
10
10
9
10
10
10
10
10
9
19
18
10
9
10
10
10
10
9
9
10
9
15
14
10
9
10
10
10
9
11
10
10
10
10
10
22
22
10
10
10
9
10
10
10
10
15
15
10
10
10
10
10
10
10
10
10
10
10
10
12
11
10
10
10
10
10
10
10
10

...

11
10
10
10
9
11
11
10
10
14
14
10
10
10
10
8
7
10
9
10
10
10
10
10
10
13
12
10
9
10
10
10
9
8
8
10
10
10
9
10
9
10
10
10
10
14
13
10
10
6
5
10
9
10
9
14
14
6
5
15
15
12
12
9
8
10
9
10
10
15
15
12
12
12
12
10
9
10
10
8
7
10
9
10
10
10
9
15
14
10
9
10
9
10
9
10
9
12
11
10
9
12
11
10
10
10
9
4
3
10
9
15
14
10
9
10
10
10
10
11
10
10
9
2
1
10
10
15
15
10
10
4
3
11
11
10
10
10
9
11
11
10
10
10
10
10
10
10
9
15
15
10
9
10
10
10
10
10
9
10
9
10
10
13
13
10
10
10
10
10
9
10
9
10
9
12
11
10
10
10
10
10
9
10
9
10
10
10
9
10
9
10
9
10
10
10
9
10
9
10
9
8
8
15
14
10
10
10
9
15
14
10
10
10
10
10
10
10
10
10
9
10
10
10
10
11
11
11
10
10
10
12
11
10
9
4
3
6
6
10
9
10
9
8
7
10
10
9
8
10
9
10
10
10
10
4
3
10
9
10
10
10
10
10
10
10
9
10
9
10
9
10
9
10
9
10
10
12
11
15
14
15
14
10
9
11
10
10
9
10
9
15
15
10
10
10
10
13
12
10
10
15
15
10
9
10
9
10
9
10
9
10
10
10
10
10
10
10
9
13
13
10
9
10
9
6
5
10
9
10
10
4
3
15
14
10
9
10
9
143
142
10
10
10
9
10
10
15
15
10
10
10
10
10
10
8
7
10
9
10
9
10
9
10
9
9
9
10

10
10
10
10
10
9
10
10
8
7
10
9
10
10
10
10
10
9
10
9
10
10
10
10
10
9
10
10
15
15
10
10
10
10
10
9
10
10
10
9
10
10
15
14
10
9
12
12
10
10
10
9
15
14
10
10
10
9
14
14
10
10
10
10
10
10
10
9
10
10
10
10
10
9
10
9
10
9
10
10
10
9
10
10
12
11
15
14
10
10
10
10
15
15
10
9
13
13
10
10
10
9
10
10
10
10
10
10
10
10
10
10
10
10
9
8
10
10
10
9
10
10
10
9
10
9
10
10
10
9
10
9
15
14
10
10
10
9
10
10
15
14
10
10
10
9
10
10
15
15
15
14
10
9
10
9
10
10
10
10
10
10
21
20
10
9
10
9
10
10
15
14
10
10
10
10
10
9
14
13
10
9
10
10
10
10
10
10
10
9
10
10
10
9
10
9
15
15
10
9
10
9
8
7
10
10
10
10
15
15
10
10
10
9
14
13
10
9
15
15
15
15
10
10
10
10
10
9
12
12
10
9
10
10
10
9
10
10
11
10
15
15
10
10
10
10
10
9
15
15
10
10
10
9
14
13
10
10
10
9
10
9
10
10
10
10
10
10
15
15
10
10
10
10
15
15
17
16
10
9
10
10
10
10
10
10
10
9
8
7
10
9
10
10
10
9
15
14
10
10
15
14
10
9
10
9
8
7
11
11
11
11
10
10
10
10
10
10
19
19
14
13
10
9
10
10
10
10
8
7
10
9
10
10
12
12
10
10
10
9
10
9
10
10
10
10
10
9
10
10
10
10
10
10
10
9


10
10
10
10
9
10
10
10
10
10
10
11
10
15
15
8
7
10
10
10
10
10
10
10
9
4
3
6
5
8
7
6
5
10
9
11
11
10
10
12
12
10
9
10
10
6
5
10
10
10
10
10
9
10
9
10
9
10
9
10
9
10
9
10
9
15
15
11
11
10
9
6
5
10
10
17
16
10
10
10
9
10
10
10
10
10
9
10
10
10
10
15
14
10
10
11
10
10
10
10
10
20
20
10
9
10
9
8
7
8
7
10
9
18
17
15
15
10
10
15
14
10
10
12
11
10
10
10
10
10
9
6
6
10
10
9
8
10
10
10
10
4
3
2
1
10
10
15
15
17
16
10
10
10
10
12
11
11
10
10
9
10
10
10
10
10
9
10
9
10
10
10
10
10
10
10
10
10
10
10
10
6
5
10
10
10
10
9
8
10
9
11
11
10
10
10
9
10
9
10
10
10
9
10
10
10
9
10
9
10
9
8
7
10
10
10
10
10
10
8
7
15
14
10
9
10
9
10
10
10
9
6
6
10
10
15
14
10
10
6
5
9
8
11
11
8
7
12
11
12
11
11
10
16
15
10
10
10
10
8
8
15
15
10
10
3
2
10
10
10
9
10
9
10
10
13
12
8
7
10
10
10
10
10
9
10
9
10
9
15
15
10
9
10
9
10
10
10
10
12
12
10
10
10
10
12
12
10
9
10
9
10
10
10
10
9
9
10
10
8
7
10
9
10
9
10
10
12
11
10
9
8
7
10
10
22
21
10
10
10
10
8
7
10
9
10
9
10
9
10
10
10
9
10
9
10
9
12
12
10
10
12
12
10
10
10
9
17
16

9
10
9
14
14
10
9
10
10
10
10
10
10
10
10
10
10
10
10
10
9
10
10
10
10
10
10
10
10
10
9
10
9
10
10
15
15
10
9
10
9
10
9
10
10
10
10
8
7
10
10
10
10
10
10
10
10
19
19
10
9
10
10
11
10
10
9
10
10
10
10
4
3
10
10
10
10
10
9
14
13
10
10
10
10
10
9
15
15
10
10
10
9
6
6
4
3
10
10
10
10
10
9
10
9
15
14
10
10
10
10
10
10
8
8
10
10
10
9
10
9
10
10
10
9
13
13
10
9
10
10
10
10
10
10
10
10
10
10
13
13
10
10
10
10
10
9
10
10
10
10
10
9
10
10
10
10
10
9
10
9
10
9
10
9
6
6
10
9
10
10
11
10
10
10
10
9
10
9
10
10
10
10
7
6
8
7
11
10
10
10
10
9
10
9
10
10
10
9
10
10
10
10
10
10
10
10
6
5
10
10
10
9
10
10
10
10
10
9
10
10
10
9
3
2
13
12
10
10
15
14
10
10
15
14
6
5
10
10
10
9
10
10
10
10
10
10
5
4
10
10
14
13
10
10
10
9
14
13
8
7
10
10
10
9
10
10
14
14
10
9
10
10
15
15
10
9
10
10
10
10
10
10
10
9
9
9
12
11
10
10
11
11
10
10
10
9
10
10
10
10
10
10
10
9
10
10
15
14
10
9
10
9
10
10
10
10
10
9
10
10
10
9
6
5
10
9
10
9
10
10
10
9
10
9
10
10
15
14
10
10
10
10
10
10
4
3
10
9
10
10
10
10
10
10
10
9
11
10
10
9
10


15
14
10
10
39
38
69
68
50
49
52
51
41
41
21
21
142
140
157
156
25
25
117
116
131
130
70
69
119
118
56
55
21
20
21
20
36
36
100
99
69
68
22
21
33
32
10
8
57
56
98
97
70
70
22
22
49
48
16
15
52
52
35
34
55
54
32
32
79
78
33
32
86
85
20
19
16
16
49
47
38
37
35
34
48
47
18
17
33
32
49
48
45
44
26
26
2
2
2
2
1
1
58
57
56
55
13
13
95
94
48
48
24
23
20
20
19
18
16
15
26
25
35
35
17
16
48
48
10
8
44
43
22
21
19
19
38
37
8
8
24
24
35
34
42
41
38
37
33
32
62
61
10
9
16
16
38
37
12
12
4
3
32
32
44
44
15
14
22
22
43
42
27
26
22
21
16
15
10
10
10
9
16
15
10
10
14
14
22
21
19
19
21
21
10
9
8
8
10
10
10
10
10
10
10
9
10
9
CPU times: user 19.1 s, sys: 3.76 s, total: 22.9 s
Wall time: 21.5 s


In [158]:
top_given_names.loc[0][7]

"M'HEMED,MOUHAMET,MEHMET,MHAMAD,MOHAMID,MUHAMMID,MOOHAMMED,MOCHEMAD,MOOHED,MEHAMMED,MOHEMMED,MHAMMED,MOUHEMED,MOWHAMMAD,MOHAMMOD,MOHAMUD,MUHEMED,MOHAMET,MUCHAMMAD,MOCHAMET,MOOHAMMAD,M'HAMAD,MHEMMED,MOCHAMMED,MOHD,MUHAMMED,M'HEMMED,MOKHAMED,MUHD,MUHAMID,MOKHAMAT,MUKHAMET,MOCHAMED,MOUHEMAD,MUKHAMMET,MUHAMIT,MUHAMMD,MUHEMAD,MUHEMMED,MOUHAMAD,MHAMMAD,MOUHAMID,MEHMED,MUKHAMMED,MUHAMAD,MOOHAMED,MOUHAMMAD,MOHAMED,MOUKHAMED,MAHAMMAD,MOHEMAT,MHEMED,MEHAMED,MOHEMED,MUHAMEED,MOHAMEET,MUHAMED,MUKHAMAT,MOHAMMED,MOHAMMET,MHEMAD,MUHEMET,MOOHAMAD,MHAMED,MOUHAMED,MOHAMMEED,MOUCHAMED,MUHEMMET,MUHMMAD,MUHAMMUD,MOHMED,MUKHAMMAT,MOUHAMMED,MOUHEMET,MOOHAMET,MOUKHAMMED,MUHAMEET,MUKHAMED,MOHAMMUD,MUHEMMAD,MOKHAMET,M'HAMED,MOKHAMMAD,MOHAMOOD,MOHAMAD,MUCHAMET,MUCHAMAT,MUHAMMAT,MOUHAMAT,MOOHAMID,MOHAMMIT,MOHEMMAD,MUHAMUD,MOHEMMET,MUCHAMMED,MIHAMAD,MOHAMMID,MOHAMD,MOUHAMMET,MOHAMMAT,MUKHAMMAD,MAHAMAD,MOHAMEED,MWAHAMD,MOHAMAT,M'HAMMAD,MUKHAMAD,MUHAMMET,MOHAMMD,MOKHAMMED,MUHAMMEED,MUHAMMAD,MOWHAMMED,MUHAMET,MUHMMED